In [ ]:
import cv2
import numpy as np

#############################
widthImg = 640
heightImg = 480
#############################

video = cv2.VideoCapture(0)

def preProcessing(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (5,5), 1)
    imgCanny = cv2.Canny(imgBlur, 200, 200)
    kernel_size = np.ones((2,2), np.uint8)
    imgDialation = cv2.dilate(imgCanny, kernel_size, iterations = 2) 
    imgEroded = cv2.erode(imgDialation, kernel_size, iterations = 1) 
    
    return imgEroded
        

def getContours(img):
    biggest = np.array([])
    maxArea = 0
    
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        if area > (0.1 * img.size):
#             cv2.drawContours(imgContour, cnt, -1, (255,0,0), 3)
            perimeter = cv2.arcLength(cnt, closed = True)
            approx = cv2.approxPolyDP(cnt, 0.01*perimeter, closed = True)
            if area > maxArea and len(approx) == 4:
                biggest = approx
                maxArea = area
    
    cv2.drawContours(imgContour, biggest, -1, (255,0,0), 20)
    return biggest
            
            
def reorder(myPoints):
    myPoints = myPoints.reshape(4,2)
    myPointsNew = np.zeros((4,1,2),np.int32)
    add = myPoints.sum(1)
    diff = np.diff(myPoints, axis=1)
    
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    myPointsNew[1] = myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    
    return myPointsNew
     
        
def getWarp(img, biggest):
    biggest = reorder(biggest)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0,0], [widthImg,0], [0,heightImg], [widthImg, heightImg]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgOutput = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
    
    imgCropped = imgOutput[20:imgOutput.shape[0]-20, 20:imgOutput.shape[1]-20]
    
    return imgCropped
    
        
        
    
while True:
    _, image = video.read()
    image = cv2.resize(image, (widthImg, heightImg))
    print(image.shape)
    imgContour = image.copy()
    imgEroded = preProcessing(image)
    
    biggest = getContours(imgEroded)
    if biggest.size !=0:
        imgWarped = getWarp(image, biggest)
        cv2.imshow("warp", imgWarped)
        cv2.imshow("video", imgContour)
    else:
        cv2.imshow("video", imgContour)
        cv2.imshow("warp", imgContour)
        
     
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        video.release()
        break